# LSTM. 8 featerow - mag, x, y, depth, dist, distance, plate, days_diff

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

2024-01-15 03:00:12.253823: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 03:00:12.253887: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 03:00:12.256548: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 03:00:12.494399: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 03:00:14.212662: W tensorflow/compiler/

In [2]:
ds_train = tf.data.Dataset.load("../data/ds_train_1_16/")
ds_val = tf.data.Dataset.load("../data/ds_val_1_16/")
ds_test = tf.data.Dataset.load("../data/ds_test_1_16/")

2024-01-15 03:00:18.247265: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-15 03:00:18.576295: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-15 03:00:18.576428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-15 03:00:18.583749: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-15 03:00:18.583889: I tensorflow/compile

In [3]:
ds_train.cardinality().numpy(), ds_val.cardinality().numpy(), ds_test.cardinality().numpy()

(3299048, 465661, 101326)

In [4]:
BATCH_SIZE = 1024
BLOCK_SIZE = 16

In [5]:
def fix_x_and_y(features, context, labels):
    return features, labels

In [6]:
ds_train = ds_train.map(fix_x_and_y)
ds_val = ds_val.map(fix_x_and_y)
ds_test = ds_test.map(fix_x_and_y)

In [7]:
for x, y in ds_train.batch(BATCH_SIZE).take(1):
    print(x.shape)
    print(x[:,:,:-3].shape, x[:,:,-3].shape, x[:,:,-2].shape, x[:,:,-1].shape)
    print(y.shape)

(1024, 16, 9)
(1024, 16, 6) (1024, 16) (1024, 16) (1024, 16)
(1024, 1)


In [8]:
# ys = []
# for x, y in ds_train.batch(BATCH_SIZE):
#     ys.append(y.numpy())
# ys = np.concatenate(ys)
# np.bincount(tf.cast(ys, tf.int32).numpy().flatten())

array([3098250,  200798])

In [9]:
#neg, pos = np.bincount(tf.cast(ys, tf.int32).numpy().flatten())
neg, pos = 3098250, 200798
total = neg + pos
total, pos, neg

(3299048, 200798, 3098250)

In [10]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(2000, seed=SEED).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [11]:
@tf.keras.saving.register_keras_serializable()
class MyModel(tf.keras.Model):
    def __init__(self, n_embed):
        super(MyModel, self).__init__()
        self.embed_plate = tf.keras.layers.Embedding(32, n_embed // 9, name="embed_plate")
        self.embed_dd = tf.keras.layers.Embedding(20, n_embed // 9, name="embed_dd")
        self.embed_magtype = tf.keras.layers.Embedding(20, n_embed // 9, name="embed_magtype")
        self.embed = tf.keras.layers.Dense(n_embed // 9 * 6, name="embed_continous")
        self.lstm1 = tf.keras.layers.LSTM(n_embed, return_sequences=True, name="lstm1")
        self.lstm2 = tf.keras.layers.LSTM(n_embed, name="lstm2")
        self.dense1 = tf.keras.layers.Dense(n_embed, activation='relu', name="pre_classifier")
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid', name="classifier")
        self.conc = tf.keras.layers.Concatenate(axis=-1, name="concatenate")

    def build_graph(self):
        x = [tf.keras.layers.Input(shape=(BLOCK_SIZE, 6), batch_size=None, name="input_continous"),
            tf.keras.layers.Input(shape=(BLOCK_SIZE), batch_size=None, name="input_plate"),
            tf.keras.layers.Input(shape=(BLOCK_SIZE), batch_size=None, name="input_dd"),
            tf.keras.layers.Input(shape=(BLOCK_SIZE), batch_size=None, name="input_magtype")]
        return tf.keras.Model(inputs=x, outputs=self.call(x))

    def call(self, x):
        cont, plate, dd, magtype = x[:,:,:-3], x[:,:,-3], x[:,:,-2], x[:,:,-1]
        x1 = self.embed(cont)
        x2 = self.embed_plate(plate)
        x3 = self.embed_dd(dd)
        x4 = self.embed_magtype(magtype)
        x = self.conc([x1, x2, x3, x4])
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [12]:
model = MyModel(135)

In [13]:
model.build(input_shape=(None, BLOCK_SIZE, 9))
model.count_params()

312886

In [14]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
            optimizer='adam',
            metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [15]:
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}
history = model.fit(ds_train, epochs=20, validation_data=ds_val, class_weight=class_weight)

Epoch 1/20


2024-01-15 03:06:26.288528: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 218 of 2000
2024-01-15 03:06:36.307175: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 437 of 2000
2024-01-15 03:06:46.310402: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 657 of 2000
2024-01-15 03:07:06.292097: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1110 of 2000
2024-01-15 03:07:16.318478: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1341 of 2000
2024-01-15 03:07:36.285599: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take 

3222/3222 [==============================] - 260s 50ms/step - loss: 0.4703 - accuracy: 0.8241 - recall: 0.7492 - precision: 0.2211 - val_loss: 0.3937 - val_accuracy: 0.8636 - val_recall: 0.4557 - val_precision: 0.2168
Epoch 2/20


2024-01-15 03:10:41.934316: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 232 of 2000
2024-01-15 03:10:51.951482: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 463 of 2000
2024-01-15 03:11:11.938628: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 926 of 2000
2024-01-15 03:11:21.958969: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1157 of 2000
2024-01-15 03:11:41.953680: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1611 of 2000


   1/3222 [..............................] - ETA: 78:02:20 - loss: 1.0106 - accuracy: 0.5195 - recall: 0.8065 - precision: 0.2475

...

3222/3222 [==============================] - 343s 81ms/step - loss: 0.2704 - accuracy: 0.8723 - recall: 0.8938 - precision: 0.3098 - val_loss: 0.4504 - val_accuracy: 0.8477 - val_recall: 0.5450 - val_precision: 0.2149
Epoch 20/20


2024-01-15 04:40:34.226303: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 246 of 2000
2024-01-15 04:40:44.259970: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 483 of 2000
2024-01-15 04:41:04.232878: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 967 of 2000
2024-01-15 04:41:14.256417: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1207 of 2000
2024-01-15 04:41:34.232719: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1691 of 2000
2024-01-15 04:41:44.240960: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take 

   1/3222 [..............................] - ETA: 74:20:31 - loss: 0.0289 - accuracy: 0.9912 - recall: 0.0000e+00 - precision: 0.0000e+00

2024-01-15 04:41:47.106591: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


3222/3222 [==============================] - 344s 81ms/step - loss: 0.2631 - accuracy: 0.8747 - recall: 0.8996 - precision: 0.3147 - val_loss: 0.4565 - val_accuracy: 0.8457 - val_recall: 0.5069 - val_precision: 0.2035


In [16]:
model.evaluate(ds_test)

99/99 [==============================] - 5s 50ms/step - loss: 0.2992 - accuracy: 0.8675 - recall: 0.7925 - precision: 0.1226


[0.2991568148136139,
 0.8675167560577393,
 0.792469322681427,
 0.12262872606515884]

In [17]:
model.save("../models/model_v6_cw.keras")

In [18]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v6_cw.csv", index=False)